In [1]:
import pandas as pd
import re
import random
# itertools — Functions creating iterators for efficient looping
# The module standardizes a core set of fast, memory efficient tools that are useful by themselves or in combination. 
from itertools import product,combinations


[itertools](https://docs.python.org/3/library/itertools.html)

<h2>TODO</h2>

### Patterns para uso

In [2]:
# Possíveis variáveis de classe
agents_dict = {
    '1S': ['EU'],
    '2S': ['TU'], 
    '3S': ['ELE', 'ELA'],
    '1P': ['NÓS'],
    '2P': ['VÓS'],
    '3P': ['ELES', 'ELAS']
}

pronouns_dict = {
    '1S': ['ME'],
    '2S': ['TE'],
    '3S': ['LHE'],
    '1P': ['NÓS'],
    '2P': ['VÓS'],
    '3P': ['LHES']
}
regex_latin ='[A-ZÁÉÍÓÚÀÂÊÔÃÕÜÇa-záéíóúàâêôãõüç]+'

#filtro para GI pattern
gi_verb_pattern = '[1-3][SP]_('+regex_latin+')_[1-3][SP]'
gi_pattern = '[1-3][SP]_'+regex_latin+'_[1-3][SP]'

# agentes
agent_pattern = r'\b(EU|TU|ELE|ELA|NÓS|VÓS|ELES|ELAS)\b'

# verbos
verb_pattern = r'\b({0}R)\b'.format(regex_latin)
# pronome obliquo atono
pronoun_pattern = r'\b(ME|TE|LHE|VOS|NOS|LHES)\b'

pattern_agent_verb = "%s %s %s" %(agent_pattern, verb_pattern, agent_pattern)
pattern_pronoun_verb = "%s %s %s" %(agent_pattern, pronoun_pattern, verb_pattern)


### Detectar os padrões das frases

In [20]:
def find_pattern(phrase):
    '''Find pattern in phrase, return 2 list with patterns found'''

    # resultado do regex para o padrão [agente -> verbo -> receptor]
    search_pattern_agent_verb = re.findall(pattern_agent_verb, phrase)
  
    # resultado do regex para o padrão [agente -> pronome -> verbo]
    search_pattern_pronoun_verb = re.findall(pattern_pronoun_verb, phrase)
    
    return search_pattern_agent_verb, search_pattern_pronoun_verb

#### Cria uma novas pattern  [agente -> verbo -> receptor] e [agente -> pronome -> verbo]

In [21]:
def new_patterns(pattern_gr,verb_gi,**kwargs):
    '''cria uma novas frases a partir de um pattern [agente -> verbo -> receptor]
    ou [agente -> pronome -> verbo]'''
    
    patterns_gr = []
    patterns_gi = []
    t_type = kwargs.get('norm')
    
    if t_type:
        verb = pattern_gr[1]
        
        # combinação com todos as possibilidades do dicionário
        combinations = product(agents_dict.keys(), agents_dict.keys(), repeat=1)

        for item in combinations:
            # criação de frases do tipo [EU, TU ...]
            # os for's são necessários nos casos de [ELE, ELA, ELES ELAS]
            for index_1, item_1  in enumerate(agents_dict[item[0]]):

                for index_2, item_2 in enumerate(agents_dict[item[1]]):

                    gr = f'{item_1} {verb} {item_2}'
                    gi = f'{item[0]}_{verb_gi}_{item[1]}'
                    patterns_gr.append(gr)
                    patterns_gi.append(gi)
    else:
        verb = pattern_gr[2]
        
        # combinação com todos as possibilidades do dicionário pronouns_dict
        combinations = product(agents_dict.keys(), pronouns_dict.keys(), repeat=1)

        for item in combinations:
            # criação de frases do tipo [EU, TU ...]
            # os for's são necessários nos casos de [ELE, ELA, ELES ELAS]
            for index_1, item_1  in enumerate(agents_dict[item[0]]):

                for index_2, item_2 in enumerate(pronouns_dict[item[1]]):

                    gr = f'{item_1} {item_2} {verb}'
                    gi = f'{item[0]}_{verb_gi}_{item[1]}'
                    patterns_gr.append(gr)
                    patterns_gi.append(gi)
    
    return patterns_gr,patterns_gi
    

### Monta nova frase a partir um padrão já criando anteriormente 

Transforma padrão [agente -> verbo -> receptor] para string

In [22]:
def for_string(*args,**kwargs):
    '''Transformando a padrão em [agente -> verbo -> receptor] ou [agente -> pronome -> verbo]
        em string para pesquisar a parte de trás e da frente do padrão'''
    
    if kwargs.get('agent_verb'):
        search_pattern = kwargs.get('agent_verb')
        
    elif kwargs.get('pronoun_verb'):
        search_pattern = kwargs.get('pronoun_verb')
        
    strings = []
    for pattern in search_pattern: 
        line = ''
        for index, part in enumerate(pattern):
            if not index:
                line +=part
            else:
                line += ' '+part
        strings.append(line)
        
    return strings

Monta nova frase a partir um padrão já criando anteriorment

In [23]:
def assembly_phrase(phrase,*args,**kwargs):
    '''Monta a frase dado [agente -> verbo -> receptor] ou [agente -> pronome -> verbo]'''
 
    new_phrase_gr = phrase[0]
    new_phrase_gi = phrase[1]
    
    combination_gr = kwargs.get('combination_gr')
    combination_gi = kwargs.get('combination_gi')
    
    if kwargs.get('agent_verb'):
        search_pattern = kwargs.get('agent_verb')    
        pstr = for_string(agent_verb = search_pattern)# Transforma search_pattern em string
    
    else:    
        search_pattern = kwargs.get('pronoun_verb')
        pstr = for_string(pronoun_verb = search_pattern)# Transforma search_pattern em string
    
    try:
        gi_verbs = re.findall(gi_pattern,new_phrase_gi)# Retira verbo do lado GI
        for i, part_string in enumerate(pstr):
            
            new_phrase_gr = re.sub(part_string,combination_gr[i],new_phrase_gr)
            new_phrase_gi = re.sub(gi_verbs[i],combination_gi[i],new_phrase_gi)
    
    except Exception as e:
        print(f'Erro assembly_phrase regex.\n',e)
    
    return (new_phrase_gr,new_phrase_gi)
    
    

# Main

Já é possível criar as frases GR para os dois padrões descritos como: [agente -> verbo -> receptor] e [agente -> pronome -> verbo]

In [36]:
def augmentation(tupla):
    new_patterns_gr = []
    new_patterns_gi =[]
    new_p = []
    
    phrase = tupla[0]

    # [agente -> verbo -> receptor] and [agente -> pronome -> verbo]
    search_pattern_agent_verb, search_pattern_pronoun_verb = find_pattern(phrase)
    
    gi_verbs = re.findall(gi_verb_pattern,tupla[1])
    
    # Shuffle nos pattern encontradas para para retirada de apenas 2 de forma aleatória,
    # porque para cada padrão é gerada 64 frases, logo é 64 ^ NumeroDePatterns
    random.shuffle(search_pattern_agent_verb)
    random.shuffle(search_pattern_agent_verb)
    
    search_pattern_agent_verb = search_pattern_agent_verb[:2]
    search_pattern_pronoun_verb = search_pattern_pronoun_verb[:2]
    
    '''[agente -> verbo -> receptor]''' 
    if search_pattern_agent_verb:
        for i, patterns_in_phrase in enumerate(search_pattern_agent_verb):
            gr,gi = new_patterns(patterns_in_phrase,gi_verbs[i],norm=1)
            
            new_patterns_gr.append(gr)
            new_patterns_gi.append(gi)          
            
    '''[agente -> pronome -> verbo]'''
    if search_pattern_pronoun_verb:
        for i, patterns_in_phrase in enumerate(search_pattern_pronoun_verb):
            gr,gi = new_patterns(patterns_in_phrase,gi_verbs[i],norm=0)
            
            new_patterns_gr.append(gr)
            new_patterns_gi.append(gi)
        
    #combinação dos pattern das frases geradas
    combination_patterns = product(*new_patterns_gr, repeat=1)
    combination_patterns_gi = product(*new_patterns_gi, repeat=1)
    
    for cont, item_gr in enumerate(combination_patterns):
        item_gi = next(combination_patterns_gi)

        if search_pattern_pronoun_verb:
            new_p.append(assembly_phrase(tupla,
                                         agent_verb = search_pattern_pronoun_verb,
                                         combination_gr = item_gr,
                                         combination_gi = item_gi))
        if search_pattern_agent_verb:
            new_p.append(assembly_phrase(tupla,
                                     agent_verb = search_pattern_agent_verb,
                                     combination_gr = item_gr,
                                     combination_gi = item_gi)) 
    # Shuffle para retornar 50 frases aleatórias
    random.shuffle(new_p)
    
    return new_p

In [37]:
#TESTES

In [40]:
phrases = [('EU TE MOSTRAR CIDADE [PONTO]', '1S_MOSTRAR_2S CIDADE [PONTO]'),
           ('ELE PEDIR ELA SE ELA CONHECER [PONTO]', '3S_PERGUNTAR_3S CONHECER [PONTO]')]
new_phrases = []
for tupla in phrases:
    
    new_phrases.append(augmentation(tupla))

In [41]:
new_phrases[0][:15]

[('EU VÓS MOSTRAR CIDADE [PONTO]', '1S_MOSTRAR_2P CIDADE [PONTO]'),
 ('TU TE MOSTRAR CIDADE [PONTO]', '2S_MOSTRAR_2S CIDADE [PONTO]'),
 ('ELES LHES MOSTRAR CIDADE [PONTO]', '3P_MOSTRAR_3P CIDADE [PONTO]'),
 ('VÓS TE MOSTRAR CIDADE [PONTO]', '2P_MOSTRAR_2S CIDADE [PONTO]'),
 ('ELAS NÓS MOSTRAR CIDADE [PONTO]', '3P_MOSTRAR_1P CIDADE [PONTO]'),
 ('ELAS TE MOSTRAR CIDADE [PONTO]', '3P_MOSTRAR_2S CIDADE [PONTO]'),
 ('ELE LHES MOSTRAR CIDADE [PONTO]', '3S_MOSTRAR_3P CIDADE [PONTO]'),
 ('EU NÓS MOSTRAR CIDADE [PONTO]', '1S_MOSTRAR_1P CIDADE [PONTO]'),
 ('TU VÓS MOSTRAR CIDADE [PONTO]', '2S_MOSTRAR_2P CIDADE [PONTO]'),
 ('TU LHES MOSTRAR CIDADE [PONTO]', '2S_MOSTRAR_3P CIDADE [PONTO]'),
 ('VÓS NÓS MOSTRAR CIDADE [PONTO]', '2P_MOSTRAR_1P CIDADE [PONTO]'),
 ('NÓS LHES MOSTRAR CIDADE [PONTO]', '1P_MOSTRAR_3P CIDADE [PONTO]'),
 ('VÓS LHES MOSTRAR CIDADE [PONTO]', '2P_MOSTRAR_3P CIDADE [PONTO]'),
 ('NÓS LHE MOSTRAR CIDADE [PONTO]', '1P_MOSTRAR_3S CIDADE [PONTO]'),
 ('TU NÓS MOSTRAR CIDADE [PONTO]',

In [28]:
len(new_phrases[1])

64

## Testando com dados reais

In [29]:
test_data = pd.read_csv('test_data/gr_gi.csv')

In [30]:
test_data.columns = ['gr','gi']

In [31]:
# Pegando as frases que possuem a necessidade de augmentation
# [agente -> verbo -> receptor] and [agente -> pronome -> verbo]
cont = 0
phrases_agent_verb = []
phrases_pronoun_verb = []
for i,phrase in enumerate(test_data['gr']):
    
    try:
        search_pattern_agent_verb, search_pattern_pronoun_verb = find_pattern(phrase)

        if search_pattern_agent_verb :
            tpl = (test_data['gr'][i],test_data['gi'][i])
            phrases_agent_verb.append(tpl)

            cont+=1
        if search_pattern_pronoun_verb:
            tpl = (test_data['gr'][i],test_data['gi'][i])
            phrases_pronoun_verb.append(tpl)
            cont +=1
        #if cont >= 10:
            #break
    except Exception as e:
        print(e)

expected string or bytes-like object


In [32]:
# Total de frases [agente -> verbo -> receptor]
len(phrases_agent_verb)

58

In [33]:
# Total de frases [agente -> pronome -> verbo]
len(phrases_pronoun_verb)

537

In [34]:
phrases_pronoun_verb[random.randint(0,len(phrases_pronoun_verb))]

('EU ME CULPAR [PONTO]', 'EU CULPADO [PONTO]')

In [35]:
phrases_agent_verb[random.randint(0,len(phrases_agent_verb))]

('ELES RIR NÓS [PONTO]', 'ELES RIR NÓS [PONTO]')

In [19]:
re.sub('a','b','abc')

'bbc'